<a href="https://colab.research.google.com/github/EmmaAB/Project_3/blob/main/updated_jeopardy_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 2s (118 kB/s)
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-21 16:38:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-11-21 16:38:58 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [ ]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://finalprojectjeopardy.s3.amazonaws.com/JEOPARDY_CSV.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("JEOPARDY_CSV.csv"), inferSchema=True, sep=',', quote="\"", escape="\"")

# Show DataFrame
df.show()
df.select(" Value").distinct().show()

+-----------+----------+---------+--------------------+------+--------------------+--------------------+
|Show Number|  Air Date|    Round|            Category| Value|            Question|              Answer|
+-----------+----------+---------+--------------------+------+--------------------+--------------------+
|       4680|2004-12-31|Jeopardy!|             HISTORY|  $200|For the last 8 ye...|          Copernicus|
|       4680|2004-12-31|Jeopardy!|ESPN's TOP 10 ALL...|  $200|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|2004-12-31|Jeopardy!|EVERYBODY TALKS A...|  $200|The city of Yuma ...|             Arizona|
|       4680|2004-12-31|Jeopardy!|    THE COMPANY LINE|  $200|In 1963, live on ...|          McDonald's|
|       4680|2004-12-31|Jeopardy!| EPITAPHS & TRIBUTES|  $200|Signer of the Dec...|          John Adams|
|       4680|2004-12-31|Jeopardy!|      3-LETTER WORDS|  $200|In the title of a...|             the ant|
|       4680|2004-12-31|Jeopardy!|             HISTORY|

In [ ]:
# import types
from pyspark.sql.types import StructField, StringType, IntegerType, StructType



In [ ]:
# see column names
df.columns

['Show Number',
 ' Air Date',
 ' Round',
 ' Category',
 ' Value',
 ' Question',
 ' Answer']

In [ ]:
# check count
df.count()

216930

In [ ]:
# dropna
df.dropna()

DataFrame[Show Number: int,  Air Date: string,  Round: string,  Category: string,  Value: string,  Question: string,  Answer: string]

In [ ]:
# double check count
df.count()

216930

In [ ]:
# print schema
df.printSchema()

root
 |-- Show Number: integer (nullable = true)
 |--  Air Date: string (nullable = true)
 |--  Round: string (nullable = true)
 |--  Category: string (nullable = true)
 |--  Value: string (nullable = true)
 |--  Question: string (nullable = true)
 |--  Answer: string (nullable = true)



In [ ]:
# rename the columns to have no spaces

final = df.withColumnRenamed('Show Number', 'show_number').withColumnRenamed(" Air Date", "air_date").withColumnRenamed(' Round', 'round').withColumnRenamed(" Category", "category").withColumnRenamed(' Value', 'value').withColumnRenamed(" Question", "question").withColumnRenamed(' Answer', 'answer') 
final.show()

+-----------+----------+---------+--------------------+-----+--------------------+--------------------+
|show_number|  air_date|    round|            category|value|            question|              answer|
+-----------+----------+---------+--------------------+-----+--------------------+--------------------+
|       4680|2004-12-31|Jeopardy!|             HISTORY| $200|For the last 8 ye...|          Copernicus|
|       4680|2004-12-31|Jeopardy!|ESPN's TOP 10 ALL...| $200|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|2004-12-31|Jeopardy!|EVERYBODY TALKS A...| $200|The city of Yuma ...|             Arizona|
|       4680|2004-12-31|Jeopardy!|    THE COMPANY LINE| $200|In 1963, live on ...|          McDonald's|
|       4680|2004-12-31|Jeopardy!| EPITAPHS & TRIBUTES| $200|Signer of the Dec...|          John Adams|
|       4680|2004-12-31|Jeopardy!|      3-LETTER WORDS| $200|In the title of a...|             the ant|
|       4680|2004-12-31|Jeopardy!|             HISTORY| $400|Bui

In [ ]:
# take out none

from pyspark.sql.functions import *
from pyspark.sql.functions import expr

final = final.withColumn('value', expr("regexp_replace(value, 'None', 0)"))
final.show()

final.select("value").distinct().show()


+-----------+----------+---------+--------------------+-----+--------------------+--------------------+
|show_number|  air_date|    round|            category|value|            question|              answer|
+-----------+----------+---------+--------------------+-----+--------------------+--------------------+
|       4680|2004-12-31|Jeopardy!|             HISTORY| $200|For the last 8 ye...|          Copernicus|
|       4680|2004-12-31|Jeopardy!|ESPN's TOP 10 ALL...| $200|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|2004-12-31|Jeopardy!|EVERYBODY TALKS A...| $200|The city of Yuma ...|             Arizona|
|       4680|2004-12-31|Jeopardy!|    THE COMPANY LINE| $200|In 1963, live on ...|          McDonald's|
|       4680|2004-12-31|Jeopardy!| EPITAPHS & TRIBUTES| $200|Signer of the Dec...|          John Adams|
|       4680|2004-12-31|Jeopardy!|      3-LETTER WORDS| $200|In the title of a...|             the ant|
|       4680|2004-12-31|Jeopardy!|             HISTORY| $400|Bui

In [ ]:
# drop 0

# df = df.filter((df.A != 'NA') & (df.B != 'NA') & (df.C != 'NA'))
# df.show()
# data=sc.TextFile('file1.txt')
# def dropFirstRow(index,iterator):
#      if(index==0):
#           for subIndex,item in enumerate(iterator):
#                if subIndex > 0:
#                     yield item
#      else:
#           yield iterator

# data=sc.TextFile('file1.txt')
# def dropFirstRow(index,iterator):
#      return iter(list(iterator)[1:]) if index==0 else iterator
# data=data.mapPartitionsWithIndex(dropFirstRow)

# df_orders1=df_orders.where("cust_no!=23512")

# people.filter(people.age > 30)


# final = final.filter(final.value > 0)
# final.show()

# final = final.dropna()
# final.show


+-----------+--------+-----+--------+-----+--------+------+
|show_number|air_date|round|category|value|question|answer|
+-----------+--------+-----+--------+-----+--------+------+
+-----------+--------+-----+--------+-----+--------+------+



In [ ]:
# check to see if dropped
final.count()

0

In [ ]:
# Configure date variable 
# from pyspark.sql.functions import year, month, day
from pyspark.sql.functions import to_date, month, year

final = final.withColumn('air_date',to_date(final.air_date, 'yyyy-MM-dd'))
final.printSchema()


root
 |-- show_number: integer (nullable = true)
 |-- air_date: date (nullable = true)
 |-- round: string (nullable = true)
 |-- category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)



In [ ]:
#Make Year + Month column
final.select(year(final["air_date"])).show()
final.select(month(final["air_date"])).show()


+--------------+
|year(air_date)|
+--------------+
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
|          2004|
+--------------+
only showing top 20 rows

+---------------+
|month(air_date)|
+---------------+
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
|             12|
+---------------+
only showing top 20 rows



In [ ]:
#Add year and month columns to dataframe
final = final.withColumn("year", year(final["air_date"]))
final = final.withColumn("month", month(final["air_date"]))
final.show()


+-----------+----------+---------+--------------------+-----+--------------------+--------------------+----+-----+
|show_number|  air_date|    round|            category|value|            question|              answer|year|month|
+-----------+----------+---------+--------------------+-----+--------------------+--------------------+----+-----+
|       4680|2004-12-31|Jeopardy!|             HISTORY| $200|For the last 8 ye...|          Copernicus|2004|   12|
|       4680|2004-12-31|Jeopardy!|ESPN's TOP 10 ALL...| $200|No. 2: 1912 Olymp...|          Jim Thorpe|2004|   12|
|       4680|2004-12-31|Jeopardy!|EVERYBODY TALKS A...| $200|The city of Yuma ...|             Arizona|2004|   12|
|       4680|2004-12-31|Jeopardy!|    THE COMPANY LINE| $200|In 1963, live on ...|          McDonald's|2004|   12|
|       4680|2004-12-31|Jeopardy!| EPITAPHS & TRIBUTES| $200|Signer of the Dec...|          John Adams|2004|   12|
|       4680|2004-12-31|Jeopardy!|      3-LETTER WORDS| $200|In the title of a..

In [ ]:
# take out the $ sign

import pyspark.sql.functions as f
from pyspark.sql.functions import col

values = final.withColumn('value', f.regexp_replace(f.col("value"), "[/$,]", ""));
values.show(50)

+-----------+----------+----------------+--------------------+-----+--------------------+--------------------+----+-----+
|show_number|  air_date|           round|            category|value|            question|              answer|year|month|
+-----------+----------+----------------+--------------------+-----+--------------------+--------------------+----+-----+
|       4680|2004-12-31|       Jeopardy!|             HISTORY|  200|For the last 8 ye...|          Copernicus|2004|   12|
|       4680|2004-12-31|       Jeopardy!|ESPN's TOP 10 ALL...|  200|No. 2: 1912 Olymp...|          Jim Thorpe|2004|   12|
|       4680|2004-12-31|       Jeopardy!|EVERYBODY TALKS A...|  200|The city of Yuma ...|             Arizona|2004|   12|
|       4680|2004-12-31|       Jeopardy!|    THE COMPANY LINE|  200|In 1963, live on ...|          McDonald's|2004|   12|
|       4680|2004-12-31|       Jeopardy!| EPITAPHS & TRIBUTES|  200|Signer of the Dec...|          John Adams|2004|   12|
|       4680|2004-12-31|

In [ ]:
# check print schema

final.printSchema()


root
 |-- show_number: integer (nullable = true)
 |-- air_date: date (nullable = true)
 |-- round: string (nullable = true)
 |-- category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)



In [ ]:
#convert everything to correct type
# final.withColumn("value", final["value"].cast(IntegerType()))
# final.show()

values = values.selectExpr("cast(show_number as int) show_number", "cast(round as string) round", "cast(air_date as date) air_date", "cast(category as string) category", "cast(value as int) value","cast(year as int) year","cast(month as int) month","cast(question as string) question", "cast(answer as string) answer")
values.printSchema()
values.show()


root
 |-- show_number: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- air_date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)

+-----------+---------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|show_number|    round|  air_date|            category|value|year|month|            question|              answer|
+-----------+---------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|       4680|Jeopardy!|2004-12-31|             HISTORY|  200|2004|   12|For the last 8 ye...|          Copernicus|
|       4680|Jeopardy!|2004-12-31|ESPN's TOP 10 ALL...|  200|2004|   12|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|Jeopardy!|2004-12-31|EVERYBODY TALKS A...|  200|2004|   

In [ ]:
#join columns from final dataframe with converted values in values dataframe on value string
# joined_df = (final.alias('final').join(values.alias('values'), on =final['value'] == values['value'], how = 'inner'))


In [ ]:
#full dataframe 
values.show(50)

+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|show_number|           round|  air_date|            category|value|year|month|            question|              answer|
+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|       4680|       Jeopardy!|2004-12-31|             HISTORY|  200|2004|   12|For the last 8 ye...|          Copernicus|
|       4680|       Jeopardy!|2004-12-31|ESPN's TOP 10 ALL...|  200|2004|   12|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|       Jeopardy!|2004-12-31|EVERYBODY TALKS A...|  200|2004|   12|The city of Yuma ...|             Arizona|
|       4680|       Jeopardy!|2004-12-31|    THE COMPANY LINE|  200|2004|   12|In 1963, live on ...|          McDonald's|
|       4680|       Jeopardy!|2004-12-31| EPITAPHS & TRIBUTES|  200|2004|   12|Signer of the Dec...|          John Adams|
|       4680|       Jeop

In [ ]:
# check count

values.count()

216930

In [ ]:
# final print schema
values.printSchema()

root
 |-- show_number: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- air_date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)



In [ ]:
# count show numbers
count = values.groupBy("show_number").count()
count.show(30)

+-----------+-----+
|show_number|count|
+-----------+-----+
|       4935|   61|
|       3749|   60|
|       3175|   61|
|        496|   57|
|       4900|   59|
|       2122|   61|
|       5518|   61|
|       3794|   61|
|       4519|   61|
|       4818|   60|
|       3918|   58|
|       2866|   61|
|       5300|   61|
|       5803|   61|
|       5156|   61|
|       1238|   57|
|       3475|   56|
|       4929|   56|
|       3698|   60|
|       3179|   61|
|       4158|   61|
|        737|   61|
|       2580|   61|
|       2811|   61|
|       3226|   59|
|       4190|   61|
|        897|   58|
|       5614|   61|
|       2999|   55|
|       4219|   61|
+-----------+-----+
only showing top 30 rows



In [ ]:
# count categories
ccat = values.groupBy("category").count()
ccat.show(30)

+--------------------+-----+
|            category|count|
+--------------------+-----+
|      JUST THE FACTS|    5|
|     COLLEGE MASCOTS|    8|
|           LIBRARIES|  101|
|           ACTRESSES|   77|
|              PLACES|   15|
|   CANADIAN FOOTBALL|    5|
|     LAW ENFORCEMENT|    6|
|  AYE!  IT'S IRELAND|    5|
|      HOLLYWOOD DOGS|    1|
|           PRINCETON|    5|
|             DRIVING|   15|
|        "AD"JECTIVES|    5|
|20th CENTURY WORL...|   10|
|     ON THE CALENDAR|    1|
|      A FLY CATEGORY|   10|
|       TV MINISERIES|    5|
|             "EVER"S|    5|
|18th CENTURY AMERICA|   25|
|DANCE IN THE DICT...|    5|
|CLOTHES MAKE THE ...|    5|
|  ALEXANDER HAMILTON|    5|
|  DISNEY FILM VOICES|   30|
|AFTER THE PRESIDENCY|    2|
|          TANK ARRAY|    5|
|          TV BLONDES|   10|
|      U.S. MOUNTAINS|   13|
|             "WATER"|   10|
|  THE ARABIAN NIGHTS|   14|
|     GET THE MESSAGE|    5|
|                CARS|   28|
+--------------------+-----+
only showing t

In [ ]:
# count round
round = values.groupBy("round").count()
round.show()

+----------------+------+
|           round| count|
+----------------+------+
|       Jeopardy!|107384|
|      Tiebreaker|     3|
| Final Jeopardy!|  3631|
|Double Jeopardy!|105912|
+----------------+------+



In [ ]:
# count values
final_data = values.groupBy("value").count()
final_data.show(30)

+-----+-----+
|value|count|
+-----+-----+
|  796|    1|
|  300| 8663|
|  350|    2|
|  601|    1|
|   22|    1|
|  367|    1|
| null| 8393|
|   20|    1|
|  500| 9016|
|  250|    5|
|    5|    9|
| 1600|10801|
|  900|  114|
|  800|31860|
|  100| 9029|
| 2547|    1|
| 1000|19539|
|  585|    1|
| 1200|11331|
|  400|42244|
|  750|    4|
|   50|    1|
|  200|30455|
|  600|20377|
| 2000|11243|
|  700|  203|
|    0| 3634|
+-----+-----+



In [ ]:
#drop na
final_df = values.na.drop()
final_df.show(50)

+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|show_number|           round|  air_date|            category|value|year|month|            question|              answer|
+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|       4680|       Jeopardy!|2004-12-31|             HISTORY|  200|2004|   12|For the last 8 ye...|          Copernicus|
|       4680|       Jeopardy!|2004-12-31|ESPN's TOP 10 ALL...|  200|2004|   12|No. 2: 1912 Olymp...|          Jim Thorpe|
|       4680|       Jeopardy!|2004-12-31|EVERYBODY TALKS A...|  200|2004|   12|The city of Yuma ...|             Arizona|
|       4680|       Jeopardy!|2004-12-31|    THE COMPANY LINE|  200|2004|   12|In 1963, live on ...|          McDonald's|
|       4680|       Jeopardy!|2004-12-31| EPITAPHS & TRIBUTES|  200|2004|   12|Signer of the Dec...|          John Adams|
|       4680|       Jeop

In [ ]:
#Check for null (again ugh)
check_for_null = final_df.groupBy("value").count()
check_for_null.show(30)

+-----+-----+
|value|count|
+-----+-----+
|  796|    1|
|  300| 8663|
|  350|    2|
|  601|    1|
|   22|    1|
|  367|    1|
|   20|    1|
|  500| 9016|
|  250|    5|
|    5|    9|
| 1600|10801|
|  900|  114|
|  800|31860|
|  100| 9029|
| 2547|    1|
| 1000|19539|
|  585|    1|
| 1200|11331|
|  400|42244|
|  750|    4|
|   50|    1|
|  200|30455|
|  600|20377|
| 2000|11243|
|  700|  203|
|    0| 3634|
+-----+-----+



In [ ]:
# import pandas
import pandas as pd
pandas_final = values.toPandas()

In [ ]:
# check in pandas
pandas_final.head(20)

,show_number,round,air_date,category,value,year,month,question,answer
0,4680,Jeopardy!,2004-12-31,HISTORY,200.0,2004,12,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,Jeopardy!,2004-12-31,ESPN's TOP 10 ALL-TIME ATHLETES,200.0,2004,12,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,Jeopardy!,2004-12-31,EVERYBODY TALKS ABOUT IT...,200.0,2004,12,The city of Yuma in this state has a record av...,Arizona
3,4680,Jeopardy!,2004-12-31,THE COMPANY LINE,200.0,2004,12,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,Jeopardy!,2004-12-31,EPITAPHS & TRIBUTES,200.0,2004,12,"Signer of the Dec. of Indep., framer of the Co...",John Adams
5,4680,Jeopardy!,2004-12-31,3-LETTER WORDS,200.0,2004,12,"In the title of an Aesop fable, this insect sh...",the ant
6,4680,Jeopardy!,2004-12-31,HISTORY,400.0,2004,12,Built in 312 B.C. to link Rome & the South of ...,the Appian Way
7,4680,Jeopardy!,2004-12-31,ESPN's TOP 10 ALL-TIME ATHLETES,400.0,2004,12,"No. 8: 30 steals for the Birmingham Barons; 2,...",Michael Jordan
8,4680,Jeopardy!,2004-12-31,EVERYBODY TALKS ABOUT IT...,400.0,2004,12,"In the winter of 1971-72, a record 1,122 inche...",Washington
9,4680,Jeopardy!,2004-12-31,THE COMPANY LINE,400.0,2004,12,This housewares store was named for the packag...,Crate & Barrel


In [ ]:
# take out null in pandas
import numpy as np

drop_null = pandas_final.dropna()
drop_null

,show_number,round,air_date,category,value,year,month,question,answer
0,4680,Jeopardy!,2004-12-31,HISTORY,200.0,2004,12,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,Jeopardy!,2004-12-31,ESPN's TOP 10 ALL-TIME ATHLETES,200.0,2004,12,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,Jeopardy!,2004-12-31,EVERYBODY TALKS ABOUT IT...,200.0,2004,12,The city of Yuma in this state has a record av...,Arizona
3,4680,Jeopardy!,2004-12-31,THE COMPANY LINE,200.0,2004,12,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,Jeopardy!,2004-12-31,EPITAPHS & TRIBUTES,200.0,2004,12,"Signer of the Dec. of Indep., framer of the Co...",John Adams
...,...,...,...,...,...,...,...,...,...
216925,4999,Double Jeopardy!,2006-05-11,RIDDLE ME THIS,2000.0,2006,5,This Puccini opera turns on the solution to 3 ...,Turandot
216926,4999,Double Jeopardy!,2006-05-11,"""T"" BIRDS",2000.0,2006,5,In North America this term is properly applied...,a titmouse
216927,4999,Double Jeopardy!,2006-05-11,AUTHORS IN THEIR YOUTH,2000.0,2006,5,"In Penny Lane, where this ""Hellraiser"" grew up...",Clive Barker
216928,4999,Double Jeopardy!,2006-05-11,QUOTATIONS,2000.0,2006,5,"From Ft. Sill, Okla. he made the plea, Arizona...",Geronimo


In [ ]:
# drop null again because wasn't working in pyspark
drop_null.count()

show_number    208537
round          208537
air_date       208537
category       208537
value          208537
question       208537
answer         208537
dtype: int64

In [ ]:
pandas_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   show_number  216930 non-null  int32  
 1   round        216930 non-null  object 
 2   air_date     216930 non-null  object 
 3   category     216930 non-null  object 
 4   value        208537 non-null  float64
 5   year         216930 non-null  int32  
 6   month        216930 non-null  int32  
 7   question     216930 non-null  object 
 8   answer       216930 non-null  object 
dtypes: float64(1), int32(3), object(5)
memory usage: 12.4+ MB


In [ ]:
# Configure settings for RDS Emma
mode = "append"
jdbc_url = "jdbc:postgresql://mypostgresdb.czptynze0rvd.us-east-1.rds.amazonaws.com:5432/jeopardy"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# # Write DataFrame to jeop_data table in RDS
# change final to last variable name

final.write.jdbc(url=jdbc_url, table='jeop_data', mode=mode, properties=config)